In [1]:
import pandas as pd

from project.thermal import conventional_energy_3uses
from project.input.param import generic_input
from project.model import get_inputs

Average losses area for envelope components.

In [2]:
ratio_surface = generic_input['ratio_surface']

In [3]:
output = get_inputs(variables=['buildings', 'efficiency', 'performance_insulation'], building_stock='project/input/stock/buildingstock_sdes2018_medium_3.csv')
buildings, eff, performance_insulation = output['buildings'], output['efficiency'], output['performance_insulation']

2022-11-16 17:12:31,465 - 98045 - log_ - INFO - Creating AgentBuildings object


In [4]:
buildings

In [19]:
worst_insulation = {i : max(output['buildings'].stock.index.get_level_values(i)) for i in ['Wall', 'Roof', 'Floor', 'Windows']}
worst_insulation.update({'Ventilation': 0.8})
print(worst_insulation)
best_insulation = {i : min(output['buildings'].stock.index.get_level_values(i)) for i in ['Wall', 'Roof', 'Floor', 'Windows']}
print(best_insulation)
performance_insulation.update({'Ventilation': 0.4})
print(performance_insulation)

{'Wall': 2.5, 'Roof': 2.5, 'Floor': 2.0, 'Windows': 4.3, 'Ventilation': 0.8}
{'Wall': 0.2, 'Roof': 0.2, 'Floor': 0.3, 'Windows': 1.3}
{'Wall': 0.2, 'Floor': 0.3, 'Roof': 0.1, 'Windows': 1.3, 'Ventilation': 0.4}


# Validation thermal function

This notebook tests the dispersion of space heating consumption and energy performance for best and worst performance building archetypes.

Defining boundaries building archetypes:

In [27]:
# Wall, Roof, Floor, Windows, Air rate (ventilation), Heating system
archetypes = {'Worst electricity': [i for i in worst_insulation.values()] + ['Electricity-Performance boiler'],
              'Best electricity water': [i for i in performance_insulation.values()] + ['Electricity-Heat pump water'],
            'Best electricity air': [i for i in performance_insulation.values()] + ['Electricity-Heat pump air'],

              'Worst gas': [i for i in worst_insulation.values()] + ['Natural gas-Standard boiler'],
              'Best gas': [i for i in performance_insulation.values()] + ['Natural gas-Performance boiler'],
              }

archetypes = pd.DataFrame(archetypes, index=['Wall', 'Roof', 'Floor', 'Windows', 'Ventilation', 'Heating system']).T
archetypes['Energy'] = archetypes['Heating system'].str.split('-').str[0].rename('Energy')

archetypes = pd.concat([archetypes, archetypes], keys=['Single-family', 'Multi-family'], names=['Housing type'])
#efficiency = pd.Series(envelope.index.get_level_values('Heating system')).replace(replace).set_axis(envelope.index)
efficiency = archetypes['Heating system'].replace(eff.to_dict()).set_axis(archetypes.index)

archetypes = pd.concat((archetypes, efficiency.rename('Efficiency')), axis=1)
archetypes = archetypes.set_index(['Heating system', 'Energy'], append=True)

archetypes = archetypes.droplevel(None)
archetypes = archetypes.astype({
    'Wall': 'float',
    'Roof': 'float', 'Floor': 'float', 'Windows': 'float', 'Ventilation': 'float', 'Efficiency': 'float'
})

archetypes

Wall  Roof  Floor  \
Housing type  Heating system                 Energy                           
Single-family Electricity-Performance boiler Electricity   2.5   2.5    2.0   
              Electricity-Heat pump water    Electricity   0.2   0.3    0.1   
              Electricity-Heat pump air      Electricity   0.2   0.3    0.1   
              Natural gas-Standard boiler    Natural gas   2.5   2.5    2.0   
              Natural gas-Performance boiler Natural gas   0.2   0.3    0.1   
Multi-family  Electricity-Performance boiler Electricity   2.5   2.5    2.0   
              Electricity-Heat pump water    Electricity   0.2   0.3    0.1   
              Electricity-Heat pump air      Electricity   0.2   0.3    0.1   
              Natural gas-Standard boiler    Natural gas   2.5   2.5    2.0   
              Natural gas-Performance boiler Natural gas   0.2   0.3    0.1   

                                                          Windows  \
Housing type  Heating system                 Energy                 
Single-family Electricity-Performance boiler Electricity      4.3   
              Electricity-Heat pump water    Electricity      1.3   
              Electricity-Heat pump air      Electricity      1.3   
              Natural gas-Standard boiler    Natural gas      4.3   
              Natural gas-Performance boiler Natural gas      1.3   
Multi-family  Electricity-Performance boiler Electricity      4.3   
              Electricity-Heat pump water    Electricity      1.3   
              Electricity-Heat pump air      Electricity      1.3   
              Natural gas-Standard boiler    Natural gas      4.3   
              Natural gas-Performance boiler Natural gas      1.3   

                                                          Ventilation  \
Housing type  Heating system                 Energy                     
Single-family Electricity-Performance boiler Electricity          0.8   
              Electricity-Heat pump water    Electricity          0.4   
              Electricity-Heat pump air      Electricity          0.4   
              Natural gas-Standard boiler    Natural gas          0.8   
              Natural gas-Performance boiler Natural gas          0.4   
Multi-family  Electricity-Performance boiler Electricity          0.8   
              Electricity-Heat pump water    Electricity          0.4   
              Electricity-Heat pump air      Electricity          0.4   
              Natural gas-Standard boiler    Natural gas          0.8   
              Natural gas-Performance boiler Natural gas          0.4   

                                                          Efficiency  
Housing type  Heating system                 Energy                   
Single-family Electricity-Performance boiler Electricity        0.95  
              Electricity-Heat pump water    Electricity        2.50  
              Electricity-Heat pump air      Electricity        2.50  
              Natural gas-Standard boiler    Natural gas        0.60  
              Natural gas-Performance boiler Natural gas        0.76  
Multi-family  Electricity-Performance boiler Electricity        0.95  
              Electricity-Heat pump water    Electricity        2.50  
              Electricity-Heat pump air      Electricity        2.50  
              Natural gas-Standard boiler    Natural gas        0.60  
              Natural gas-Performance boiler Natural gas        0.76

#### Calculating energy dispersion without ventilation

In [34]:
dpe, energy_primary = conventional_energy_3uses(archetypes['Wall'], archetypes['Floor'], archetypes['Roof'], archetypes['Windows'], ratio_surface, archetypes['Efficiency'], archetypes.index)
result = pd.concat((archetypes, dpe.rename('Performance'), energy_primary.rename('Energy')), axis=1).sort_index()
result

Wall  Roof  Floor  \
Housing type  Heating system                 Energy                           
Multi-family  Electricity-Heat pump air      Electricity   0.2   0.3    0.1   
              Electricity-Heat pump water    Electricity   0.2   0.3    0.1   
              Electricity-Performance boiler Electricity   2.5   2.5    2.0   
              Natural gas-Performance boiler Natural gas   0.2   0.3    0.1   
              Natural gas-Standard boiler    Natural gas   2.5   2.5    2.0   
Single-family Electricity-Heat pump air      Electricity   0.2   0.3    0.1   
              Electricity-Heat pump water    Electricity   0.2   0.3    0.1   
              Electricity-Performance boiler Electricity   2.5   2.5    2.0   
              Natural gas-Performance boiler Natural gas   0.2   0.3    0.1   
              Natural gas-Standard boiler    Natural gas   2.5   2.5    2.0   

                                                          Windows  \
Housing type  Heating system                 Energy                 
Multi-family  Electricity-Heat pump air      Electricity      1.3   
              Electricity-Heat pump water    Electricity      1.3   
              Electricity-Performance boiler Electricity      4.3   
              Natural gas-Performance boiler Natural gas      1.3   
              Natural gas-Standard boiler    Natural gas      4.3   
Single-family Electricity-Heat pump air      Electricity      1.3   
              Electricity-Heat pump water    Electricity      1.3   
              Electricity-Performance boiler Electricity      4.3   
              Natural gas-Performance boiler Natural gas      1.3   
              Natural gas-Standard boiler    Natural gas      4.3   

                                                          Ventilation  \
Housing type  Heating system                 Energy                     
Multi-family  Electricity-Heat pump air      Electricity          0.4   
              Electricity-Heat pump water    Electricity          0.4   
              Electricity-Performance boiler Electricity          0.8   
              Natural gas-Performance boiler Natural gas          0.4   
              Natural gas-Standard boiler    Natural gas          0.8   
Single-family Electricity-Heat pump air      Electricity          0.4   
              Electricity-Heat pump water    Electricity          0.4   
              Electricity-Performance boiler Electricity          0.8   
              Natural gas-Performance boiler Natural gas          0.4   
              Natural gas-Standard boiler    Natural gas          0.8   

                                                          Efficiency  \
Housing type  Heating system                 Energy                    
Multi-family  Electricity-Heat pump air      Electricity        2.50   
              Electricity-Heat pump water    Electricity        2.50   
              Electricity-Performance boiler Electricity        0.95   
              Natural gas-Performance boiler Natural gas        0.76   
              Natural gas-Standard boiler    Natural gas        0.60   
Single-family Electricity-Heat pump air      Electricity        2.50   
              Electricity-Heat pump water    Electricity        2.50   
              Electricity-Performance boiler Electricity        0.95   
              Natural gas-Performance boiler Natural gas        0.76   
              Natural gas-Standard boiler    Natural gas        0.60   

                                                         Performance  \
Housing type  Heating system                 Energy                    
Multi-family  Electricity-Heat pump air      Electricity           C   
              Electricity-Heat pump water    Electricity           A   
              Electricity-Performance boiler Electricity           G   
              Natural gas-Performance boiler Natural gas           B   
              Natural gas-Standard boiler    Natural gas           E   
Single-family Electricity-Heat pum

- The performance of the building archetypes ranges from A to G, which is the necessary range to address the policy in France
- The worst performing fossil fuel-fired multifamily buildings are not G. This is explained by using the average loss surface for each component. In a building, the first and last floors would get more and the other floors less. Here archetypes represent an average apartment in the building. This assumption centers the distribution around the mean.

#### Calculating energy dispersion with ventilation

In [35]:
dpe, energy_primary = conventional_energy_3uses(archetypes['Wall'], archetypes['Floor'], archetypes['Roof'], archetypes['Windows'], ratio_surface, archetypes['Efficiency'], archetypes.index, air_rate=archetypes['Ventilation'])
result = pd.concat((archetypes, dpe.rename('Performance'), energy_primary.rename('Energy')), axis=1).sort_index()
result

Wall  Roof  Floor  \
Housing type  Heating system                 Energy                           
Multi-family  Electricity-Heat pump air      Electricity   0.2   0.3    0.1   
              Electricity-Heat pump water    Electricity   0.2   0.3    0.1   
              Electricity-Performance boiler Electricity   2.5   2.5    2.0   
              Natural gas-Performance boiler Natural gas   0.2   0.3    0.1   
              Natural gas-Standard boiler    Natural gas   2.5   2.5    2.0   
Single-family Electricity-Heat pump air      Electricity   0.2   0.3    0.1   
              Electricity-Heat pump water    Electricity   0.2   0.3    0.1   
              Electricity-Performance boiler Electricity   2.5   2.5    2.0   
              Natural gas-Performance boiler Natural gas   0.2   0.3    0.1   
              Natural gas-Standard boiler    Natural gas   2.5   2.5    2.0   

                                                          Windows  \
Housing type  Heating system                 Energy                 
Multi-family  Electricity-Heat pump air      Electricity      1.3   
              Electricity-Heat pump water    Electricity      1.3   
              Electricity-Performance boiler Electricity      4.3   
              Natural gas-Performance boiler Natural gas      1.3   
              Natural gas-Standard boiler    Natural gas      4.3   
Single-family Electricity-Heat pump air      Electricity      1.3   
              Electricity-Heat pump water    Electricity      1.3   
              Electricity-Performance boiler Electricity      4.3   
              Natural gas-Performance boiler Natural gas      1.3   
              Natural gas-Standard boiler    Natural gas      4.3   

                                                          Ventilation  \
Housing type  Heating system                 Energy                     
Multi-family  Electricity-Heat pump air      Electricity          0.4   
              Electricity-Heat pump water    Electricity          0.4   
              Electricity-Performance boiler Electricity          0.8   
              Natural gas-Performance boiler Natural gas          0.4   
              Natural gas-Standard boiler    Natural gas          0.8   
Single-family Electricity-Heat pump air      Electricity          0.4   
              Electricity-Heat pump water    Electricity          0.4   
              Electricity-Performance boiler Electricity          0.8   
              Natural gas-Performance boiler Natural gas          0.4   
              Natural gas-Standard boiler    Natural gas          0.8   

                                                          Efficiency  \
Housing type  Heating system                 Energy                    
Multi-family  Electricity-Heat pump air      Electricity        2.50   
              Electricity-Heat pump water    Electricity        2.50   
              Electricity-Performance boiler Electricity        0.95   
              Natural gas-Performance boiler Natural gas        0.76   
              Natural gas-Standard boiler    Natural gas        0.60   
Single-family Electricity-Heat pump air      Electricity        2.50   
              Electricity-Heat pump water    Electricity        2.50   
              Electricity-Performance boiler Electricity        0.95   
              Natural gas-Performance boiler Natural gas        0.76   
              Natural gas-Standard boiler    Natural gas        0.60   

                                                         Performance  \
Housing type  Heating system                 Energy                    
Multi-family  Electricity-Heat pump air      Electricity           B   
              Electricity-Heat pump water    Electricity           A   
              Electricity-Performance boiler Electricity           G   
              Natural gas-Performance boiler Natural gas           B   
              Natural gas-Standard boiler    Natural gas           F   
Single-family Electricity-Heat pum

Adding ventilation does not change a lot the dispersion

#### Testing rounded value

In [8]:
buildings_round = buildings.copy()
buildings_round['Wall'] = buildings_round['Wall'].round(1)
buildings_round['Floor'] = buildings_round['Floor'].round(1)
buildings_round['Roof'] = buildings_round['Roof'].round(1)
buildings_round['Windows'] = buildings_round['Windows'].round(1)
buildings_round['Efficiency'] = buildings_round['Efficiency'].round(1)

dpe, energy_primary = conventional_energy_3uses(buildings_round['Wall'], buildings_round['Floor'], buildings_round['Roof'], buildings_round['Windows'], ratio_surface, buildings_round['Efficiency'], buildings_round.index, air_rate=buildings['Ventilation'])
result = pd.concat((buildings_round, dpe, energy_primary), axis=1).sort_index()
result

Wall  Roof  Floor  \
Housing type  Heating system                 Energy                           
Multi-family  Electricity-Heat pump          Electricity   0.2   0.3    0.1   
                                             Electricity   0.1   0.2    0.1   
              Electricity-Performance boiler Electricity   2.5   2.5    2.0   
              Natural gas-Performance boiler Natural gas   0.2   0.3    0.1   
                                             Natural gas   0.1   0.2    0.1   
              Natural gas-Standard boiler    Natural gas   2.5   2.5    2.0   
Single-family Electricity-Heat pump          Electricity   0.2   0.3    0.1   
                                             Electricity   0.1   0.2    0.1   
              Electricity-Performance boiler Electricity   2.5   2.5    2.0   
              Natural gas-Performance boiler Natural gas   0.2   0.3    0.1   
                                             Natural gas   0.1   0.2    0.1   
              Natural gas-Standard boiler    Natural gas   2.5   2.5    2.0   

                                                          Windows  \
Housing type  Heating system                 Energy                 
Multi-family  Electricity-Heat pump          Electricity      1.3   
                                             Electricity      0.8   
              Electricity-Performance boiler Electricity      3.0   
              Natural gas-Performance boiler Natural gas      1.3   
                                             Natural gas      0.8   
              Natural gas-Standard boiler    Natural gas      3.0   
Single-family Electricity-Heat pump          Electricity      1.3   
                                             Electricity      0.8   
              Electricity-Performance boiler Electricity      3.0   
              Natural gas-Performance boiler Natural gas      1.3   
                                             Natural gas      0.8   
              Natural gas-Standard boiler    Natural gas      3.0   

                                                          Ventilation  \
Housing type  Heating system                 Energy                     
Multi-family  Electricity-Heat pump          Electricity          0.4   
                                             Electricity          0.4   
              Electricity-Performance boiler Electricity          0.8   
              Natural gas-Performance boiler Natural gas          0.4   
                                             Natural gas          0.4   
              Natural gas-Standard boiler    Natural gas          0.8   
Single-family Electricity-Heat pump          Electricity          0.4   
                                             Electricity          0.4   
              Electricity-Performance boiler Electricity          0.8   
              Natural gas-Performance boiler Natural gas          0.4   
                                             Natural gas          0.4   
              Natural gas-Standard boiler    Natural gas          0.8   

                                                          Efficiency  0  \
Housing type  Heating system                 Energy                       
Multi-family  Electricity-Heat pump          Electricity         2.5  A   
                                             Electricity         2.5  A   
              Electricity-Performance boiler Electricity         1.0  G   
              Natural gas-Performance boiler Natural gas         0.8  B   
                                             Natural gas         0.8  A   
              Natural gas-Standard boiler    Natural gas         0.6  E   
Single-family Electricity-Heat pump          Electricity         2.5  A   
                                             Electricity         2.5  A   
              Electricity-Performance boiler Electricity         1.0  G   
              Natural gas-Performance boiler Natural gas         0.8  B   
                                             Natural gas     

In [7]:
buildings_round['Wall'].astype('float').round(1)

Housing type   Heating system                  Energy     
Single-family  Electricity-Performance boiler  Electricity    2.5
               Electricity-Heat pump           Electricity    0.2
                                               Electricity    0.1
               Natural gas-Standard boiler     Natural gas    2.5
               Natural gas-Performance boiler  Natural gas    0.2
                                               Natural gas    0.1
Multi-family   Electricity-Performance boiler  Electricity    2.5
               Electricity-Heat pump           Electricity    0.2
                                               Electricity    0.1
               Natural gas-Standard boiler     Natural gas    2.5
               Natural gas-Performance boiler  Natural gas    0.2
                                               Natural gas    0.1
Name: Wall, dtype: float64